<a href="https://colab.research.google.com/github/yfzhang3/2048/blob/main/v2_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load & Preprocess the data

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
from torchtext.data.utils import get_tokenizer
import nltk
from nltk.tokenize import word_tokenize
from challenge_dataset import ChallengeDataset
# I don't know where this notebook is so idk which directory this belongs in

ImportError: ignored

In [5]:
nltk.download('punkt')  # Download the tokenizer data (only need to do this once)

torch.manual_seed(1)  # set a manual seed for training reproducibility for random parameters

# Load training file data
training_path = 'train.txt'
with open(training_path, 'r', encoding='utf-8', errors='replace') as file:
    lines = file.readlines()

# Split data into original and corrupted at tab
data = [line.strip().split('\t') for line in lines]
print(data[:5])

# Separate original and corrupted
original_sentences = [pair[0] for pair in data]
corrupted_sentences = [pair[1] for pair in data]
print(original_sentences[:3])

# Tokenize the sentences
tokenized_sentences = []
for sentence in original_sentences:
    tokens = word_tokenize(sentence)
    tokenized_sentences.append(tokens)

print(tokenized_sentences[:3])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['En route , they pick up a seemingly-harmless hitchhiker , and continue their journey , only for their car to break down in a deserted motel on a lonely highway .', 'En route , they pick up a seemingly-harmless hitchhiker , and continue their , journey only for their car to break down in a deserted motel on a lonely highway .'], ['Jobs, however, are not created by supporting a few multinational giants, which will often strike sail at the first signs of an economic headwind, leaving their massive redundant crews to take to the lifeboats; on the contrary, jobs are created by promoting small and medium-sized businesses, which, after all, are the largest overall employers of manpower, not only in Austria.', 'Jobs, however, are not created by supporting a few multinational giants, which will often strike sail at the first signs of an economic heawdind, leaving their massive redundant crews to take to the lifeboats; on the contrary, jobs are created by promoting small and medium-sized busi

In [27]:
# split into training & validation but currently not working
custom_dataset = ChallengeDataset(training_data, word_to_ix)

train_subset = custom_dataset.train_data
val_subset = custom_dataset.val_data

NameError: ignored

###Tokenize Data

In [26]:
import torchtext
from torchtext.data.utils import get_tokenizer

# Customize hyperparameters
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100

# example REMOVE this BELOW when actually training
tokenized_sentences = [['En', 'route', ',', 'they', 'pick', 'up', 'a', 'seemingly-harmless', 'hitchhiker', ',', 'and', 'continue', 'their', 'journey', ',', 'only', 'for', 'their', 'car', 'to', 'break', 'down', 'in', 'a', 'deserted', 'motel', 'on', 'a', 'lonely', 'highway', '.'], ['Jobs', ',', 'however', ',', 'are', 'not', 'created', 'by', 'supporting', 'a', 'few', 'multinational', 'giants', ',', 'which', 'will', 'often', 'strike', 'sail', 'at', 'the', 'first', 'signs', 'of', 'an', 'economic', 'headwind', ',', 'leaving', 'their', 'massive', 'redundant', 'crews', 'to', 'take', 'to', 'the', 'lifeboats', ';', 'on', 'the', 'contrary', ',', 'jobs', 'are', 'created', 'by', 'promoting', 'small', 'and', 'medium-sized', 'businesses', ',', 'which', ',', 'after', 'all', ',', 'are', 'the', 'largest', 'overall', 'employers', 'of', 'manpower', ',', 'not', 'only', 'in', 'Austria', '.'], ['Due', 'to', 'circumstances', ',', 'he', 'is', 'forced', 'to', 'abandon', 'his', 'studies', ',', 'though', 'he', 'comes', 'first', 'in', 'his', 'school', 'in', 'the', 'board', 'exam', '.']];
# Flatten data into 1 list
all_words = [word for sentence in tokenized_sentences for word in sentence]
print(all_words[:2])

# Deriving a set from `original_sentences` to remove duplicates
vocab = set(all_words)
vocab_size = len(vocab)

# dictionary that maps words to their corresponding indices
word_to_ix = {word:ix for ix, word in enumerate(vocab)}
# dictionary that maps indices to their corresponding words
ix_to_word = {ix:word for ix, word in enumerate(vocab)}


# Creating training data
data = []
for sentence in tokenized_sentences:
  for i in range(CONTEXT_SIZE, len(sentence) - CONTEXT_SIZE):
      context = (
          sentence[i - CONTEXT_SIZE:i]  # Context words to the left
          + sentence[i + 1:i + CONTEXT_SIZE + 1]  # Context words to the right
      )
      target = sentence[i]
      data.append((context, target))

    # context = (
    #     [original_sentences[i - j - 1] for j in range(CONTEXT_SIZE)]  # Fixed: selecting words, not sentences
    #     + [original_sentences[i + j + 1] for j in range(CONTEXT_SIZE)]  # Fixed: selecting words, not sentences
    # )
    # target = original_sentences[i]
    # data.append((context, target))
print(data[:5])

['En', 'route']
[(['En', 'route', 'they', 'pick'], ','), (['route', ',', 'pick', 'up'], 'they'), ([',', 'they', 'up', 'a'], 'pick'), (['they', 'pick', 'a', 'seemingly-harmless'], 'up'), (['pick', 'up', 'seemingly-harmless', 'hitchhiker'], 'a')]


### Defining the Neural Network

In [18]:
class CBOW(nn.Module):

    # params for vocab size, embedding dem, batch size
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        context_vector = torch.sum(embeds, dim=0).view(1, -1) # Summing word embeddings
        print(context_vector.shape) # torch.Size([1, 100])
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

# functions from pytorch docs
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)  # example

tensor([66, 25, 19, 32])

### Creating & Training the Model

In [16]:
model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

#TRAINING
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        # Vector mapping each word in context to appropriate index
        context_vector = make_context_vector(context, word_to_ix)
        # 1D Tensor (1,) holding the index of the target word * len of context vector
        target_tensor = torch.tensor([word_to_ix[target]] * len(context_vector))

        # Passes context vector through model -> returns predicted log probabilities
        log_probs = model(context_vector)
        loss = loss_function(log_probs, target_tensor)
        print("loss", loss)

        # Cumulative loss over all instances in the current epoch
        # used in backprop
        total_loss += loss

    #optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    print("finish training")

Streaming output truncated to the last 5000 lines.
torch.Size([1, 100])
loss tensor(4.1141, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(4.1389, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(4.2502, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(4.2827, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(2.2257, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(4.4920, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(3.7694, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(4.4624, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(4.1798, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(4.3878, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(3.6918, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(4.4296, grad_fn=<NllLossBackward0>)
hi
torch.Size([1, 100])
loss tensor(3.6544, grad_fn=<NllLossBackward0>)
hi
torch.Size([1

### Evaluate Model

In [ ]:
# set the model to evaluation mode
model.eval()

# VALIDATATION LOOP
# Temporarily disables gradient computation to speed up calculations
with torch.no_grad():
    correct = 0
    total = 0
    for context, target in validation_data:
        # Generates vector, predicats lob probs
        context_vector = make_context_vector(context, word_to_ix)
        target_idx = word_to_ix[target]
        log_probs = model(context_vector)

        # Gets the predicted index w highest log prob
        _, predicted = log_probs.max(1)

        # Checks if prediction is correct
        if predicted.item() == target_idx:
            correct += 1
        total += 1

validation_accuracy = correct / total
print("Validation accuracy:", validation_accuracy)
